In [47]:
  # import 해줄 것들
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
from selenium import webdriver
import sys
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys

In [48]:
# 사이트 접속
url = input('위메프 URL 입력:')
driver = webdriver.Chrome('c:/py_data/chromedriver.exe')
driver.get(url)
time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

위메프 URL 입력:https://front.wemakeprice.com/deal/607935047?refer=AD


In [49]:
# 쇼핑몰 리뷰 누르기

menu_area = soup.find('ul', class_='tab_info noscript').find_all('li')
for i in range(len(menu_area)):
    menu = re.sub('[^ㄱ-ㅣ가-힣 ]','',menu_area[i].text)
    if menu == '구매후기':
        driver.find_element_by_css_selector('#_contents > div.content_main.items > div.dinfo_itemdetail > div.wrap_tab > div > ul > li:nth-child('+str(i+1)+') > a').click()   # 페이지별로 바꿔줘야한다.
        review_num = int(re.sub('[^0-9]','',menu_area[i].find('span', class_='num_review').text)) 
        break

In [50]:
point_lst=[]
doc_lst=[]
now_page = 0
next_btn_path = '#_contents > div.content_main.items > div.dinfo_itemdetail > div.detail_left > div.prd_detail > div.tab_conts.tab_review > div > div:nth-child(3) > div.review_wrap > div.paging_comm >'
next_btn = [' a:nth-child(2)',' a:nth-child(5)',' a:nth-child(6)',' a:nth-child(7)',
            ' a:nth-child(8)',' a:nth-child(9)',' a:nth-child(10)',' a:nth-child(11)',
           ' a:nth-child(12)']


for page in tqdm(range(review_num//len(soup.find_all('div',class_='area_cont'))+1)):
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    review_area = soup.find_all('div',class_='area_cont')
    point_area = soup.find_all('div', class_='area_star star')
    
    for review in review_area:
        review_txt = review.text
        doc_lst.append(re.sub('[^가-힣 ]','',review_txt).replace("포토리뷰",""))
    for point in point_area:
        point_num = point.find('strong', class_='grade').text
          point_lst.append(point_num)
        
    try:
        driver.find_element_by_css_selector(next_btn_path+next_btn[now_page]).click()
        time.sleep(2)
    except:
        pass
    
    if next_btn[now_page] == next_btn[-1]:
        now_page=0
        next_btn = [' a:nth-child(4)',' a:nth-child(4)',' a:nth-child(5)',' a:nth-child(6)',' a:nth-child(7)',
            ' a:nth-child(8)',' a:nth-child(9)',' a:nth-child(10)',' a:nth-child(11)',' a:nth-child(12)']
    else:
        now_page = now_page + 1

driver.close()        
df = pd.DataFrame({'별점':point_lst, '리뷰': doc_lst})
df.to_csv('c:/py_data/위메프 리뷰 크롤링.csv',encoding="cp949")
print(len(list(doc_lst)),'개의 게시물을 크롤링 하였습니다.')

## 코드정리

In [52]:
# import 해줄 것들
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
from selenium import webdriver
import sys
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys

# 사이트 접속
url = input('위메프 URL 입력:')
driver = webdriver.Chrome('c:/py_data/chromedriver.exe')
driver.get(url)
time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 쇼핑몰 리뷰 누르기

menu_area = soup.find('ul', class_='tab_info noscript').find_all('li')
for i in range(len(menu_area)):
    menu = re.sub('[^ㄱ-ㅣ가-힣 ]','',menu_area[i].text)
    if menu == '구매후기':
        driver.find_element_by_css_selector('#_contents > div.content_main.items > div.dinfo_itemdetail > div.wrap_tab > div > ul > li:nth-child('+str(i+1)+') > a').click()   # 페이지별로 바꿔줘야한다.
        review_num = int(re.sub('[^0-9]','',menu_area[i].find('span', class_='num_review').text)) 
        break
        
        
point_lst=[]
doc_lst=[]
now_page = 0
next_btn_path = '#_contents > div.content_main.items > div.dinfo_itemdetail > div.detail_left > div.prd_detail > div.tab_conts.tab_review > div > div:nth-child(3) > div.review_wrap > div.paging_comm >'
next_btn = [' a:nth-child(2)',' a:nth-child(5)',' a:nth-child(6)',' a:nth-child(7)',
            ' a:nth-child(8)',' a:nth-child(9)',' a:nth-child(10)',' a:nth-child(11)',
           ' a:nth-child(12)']


for page in tqdm(range(review_num//len(soup.find_all('div',class_='area_cont'))+1)):
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    review_area = soup.find_all('div',class_='area_cont')
    point_area = soup.find_all('div', class_='area_star star')
    
    for review in review_area:
        review_txt = review.text
        doc_lst.append(re.sub('[^가-힣 ]','',review_txt).replace("포토리뷰",""))
    for point in point_area:
        point_num = float(point.find('strong', class_='grade').text)
        point_lst.append(point_num)
        
    try:
        driver.find_element_by_css_selector(next_btn_path+next_btn[now_page]).click()
        time.sleep(2)
    except:
        pass
    
    if next_btn[now_page] == next_btn[-1]:
        now_page=0
        next_btn = [' a:nth-child(4)',' a:nth-child(4)',' a:nth-child(5)',' a:nth-child(6)',' a:nth-child(7)',
            ' a:nth-child(8)',' a:nth-child(9)',' a:nth-child(10)',' a:nth-child(11)',' a:nth-child(12)']
    else:
        now_page = now_page + 1

driver.close()        
df = pd.DataFrame({'별점':point_lst, '리뷰': doc_lst})
df.to_csv('c:/py_data/위메프 리뷰 크롤링.csv',encoding="cp949")
print(len(list(doc_lst)),'개의 게시물을 크롤링 하였습니다.')

위메프 URL 입력:https://front.wemakeprice.com/deal/620381629



192 개의 게시물을 크롤링 하였습니다.


In [54]:
df

,별점,리뷰
0,5.0,아디다스 매장정품 아디다스 남여공용 원래 사이즈인데 후기들을 찾아보니 크게나...
1,1.0,아디다스 매장정품 아디다스 남성 니트 트레이닝 상하의세트 배송에 문제가 생...
2,5.0,아디다스 매장정품 아디다스 남성트레이닝복세트 아디다스 트레이닝복을 이가격...
3,5.0,아디다스 매장정품 아디다스 화이트 생각보다 사이즈가 약간 크네요운동화는 늘 사...
4,5.0,아디다스 아디다스 남여공용 블랙 아이 신발로 구매했는데역시 아디다스는 사이즈 ...
...,...,...
187,5.0,아디다스 아디다스 남여공용 그레이 감사합니다 잘 신어요
188,5.0,아디다스 매장정품 아디다스 남성 니트 트레이닝 상하의세트 너무 좋아요 대박나세
189,3.0,아디다스 매장정품 아디다스 남성 니트 트레이닝 상하의세트 좋아요 좋아용 좋...
190,5.0,아디다스 매장정품 아디다스 여성 니트 트레이닝 상하의세트 너무좋아요 대박나세요


## 모범 답안

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import sys

url = input('특가대표 위메프 url을 입력하세요: ')
driver = webdriver.Chrome("C:/py_data/chromedriver.exe",)
driver.get(url)
time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
menu_area = soup.find('ul',class_='tab_info noscript').find_all("li")
for i in range(len(menu_area)):
    menu = menu_area[i].text
    menu_txt = re.sub("[^가-힣]","",menu)
    if '구매후기' in menu_txt:
        driver.find_element_by_css_selector('#_contents > div.content_main.items > div.dinfo_itemdetail > div.wrap_tab > div > ul > li:nth-child('+str(i+1)+") > a > span").click()
        time.sleep(1)
        review_num = int(re.sub("[^0-9]","",menu))
        break
review_lst = []
point_lst = []
for page in range(review_num//len(review_area)):
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    review_area = soup.find('div',class_='review_list').find_all('p',class_='post')
    point_area = soup.find_all('div',class_='area_star star')
    
    for i in range(len(point_area)):
        review = re.sub("[^가-힣 ]","",review_area[i].text)
        point = float(re.sub("[^0-9.]","",point_area[i].find('strong').text))
        
        review_lst.append(review)
        point_lst.append(point)
    
    try:
        driver.find_element_by_css_selector('#_contents > div.content_main.items > div.dinfo_itemdetail > div.detail_left > div.prd_detail > div.tab_conts.tab_review > div > div:nth-child(3) > div.review_wrap > div.paging_comm > a.ico.btn_next').click()
        time.sleep(1)
    except:
        pass
df = pd.DataFrame({"평점":point_lst,"본문":review_lst})
df.to_csv("c:/py_data/위메프 상품 리뷰.csv",encoding="cp949")
print(len(point_lst),'개의 게시물을 크롤링 하였습니다.')
driver.close()    